In [18]:
from pathlib import Path
from openai import OpenAI

# 1) Key aus Datei lesen (nur der Key in einer Zeile)
key_path = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MA\OpenAI_API_Key.txt")
api_key = key_path.read_text(encoding="utf-8").strip()

# Optional: simple Plausibilitätsprüfung
if not api_key or not any(api_key.startswith(p) for p in ("sk-", "sk-proj-")):
    raise ValueError("API-Key in der Datei wirkt ungültig (Präfix fehlt).")

# 2) Client mit Key initialisieren
client = OpenAI(api_key=api_key)

# 3) Testaufruf (Responses API, empfohlen)
resp = client.responses.create(
    model="gpt-4.1-2025-04-14",
    input="Erkläre in 2 Sätzen, was ein Zustandsautomat ist – auf Deutsch."
)
print(resp.output_text)

Ein Zustandsautomat (auch endlicher Automat genannt) ist ein mathematisches Modell, das aus einer endlichen Anzahl von Zuständen besteht und durch definierte Übergänge zwischen diesen Zuständen auf Eingaben reagiert. Er wird häufig verwendet, um Abläufe oder das Verhalten von Systemen eindeutig und formal zu beschreiben.


In [8]:
# TwinCAT: POUs/DUTs/GVLs/VISUs sammeln + ST-IO-Variablen extrahieren (Jupyter-ready)

from pathlib import Path
from collections import Counter, defaultdict
import re, json, xml.etree.ElementTree as ET

# ---------- Helpers ----------
def read_text(p: Path) -> str:
    return p.read_text(encoding="utf-8", errors="replace")

def strip_ns(xml_text: str) -> str:
    # Default-Namespaces entfernen -> XPath wird einfacher
    return re.sub(r'\sxmlns="[^"]+"', '', xml_text, count=1)

def strip_st_comments(s: str) -> str:
    # ST-Kommentare entfernen: (* ... *) und // ...
    s = re.sub(r'\(\*.*?\*\)', '', s, flags=re.S)
    s = re.sub(r'//.*', '', s)
    return s

def detect_impl_lang(impl_node):
    """Finde ST/FBD/LD/SFC/IL auch wenn ein NWL-Container dazwischen sitzt."""
    if impl_node is None:
        return None, ""
    for tag in ("ST", "FBD", "LD", "SFC", "IL"):
        n = impl_node.find(f".//{tag}")
        if n is not None:
            return tag, (n.text or "").strip()
    # Fallback: erster Child-Tagname (z. B. 'NWL')
    if list(impl_node):
        c = list(impl_node)[0]
        return c.tag, (c.text or "").strip()
    return None, ""

# ---------- IEC/ST Deklarationsparser ----------
_var_stmt_re = re.compile(
    r'^\s*([A-Za-z_]\w*)'               # Name
    r'(?:\s+AT\s+([^:]+))?'             # optional AT-Adresse
    r'\s*:\s*'                          
    r'([^:=;]+?)'                       # Typ (inkl. ARRAY[..] OF ...)
    r'(?:\s*:=\s*([^;]+?))?'            # optional Initialwert
    r'\s*;\s*$', re.M | re.S)

def _extract_var_block(text: str, scope_keyword: str) -> list[dict]:
    """
    Extrahiert Variablen aus einem Block VAR_<SCOPE> ... END_VAR.
    scope_keyword: 'INPUT' | 'OUTPUT' | 'IN_OUT' | 'GLOBAL' | 'TEMP' | etc.
    """
    txt = strip_st_comments(text)
    # Nicht-gierige Suche inkl. evtl. Zusätzen wie CONSTANT/RETAIN nach VAR_<SCOPE>
    m = re.search(rf'VAR_{scope_keyword}\b.*?\n(.*?)END_VAR', txt, flags=re.S | re.I)
    if not m:
        return []
    block = m.group(1)
    vars_ = []
    # Auf Semikolons getrimmt parsen
    for m2 in _var_stmt_re.finditer(block):
        name, at_addr, typ, init = [g.strip() if g else None for g in m2.groups()]
        vars_.append({
            "name": name,
            "address": at_addr,
            "type": re.sub(r'\s+', ' ', typ).strip(),
            "init": init.strip() if init else None
        })
    return vars_

def extract_io_from_declaration(declaration: str) -> dict:
    """Liest IO-Variablen aus der ST-Deklaration."""
    return {
        "inputs": _extract_var_block(declaration, "INPUT"),
        "outputs": _extract_var_block(declaration, "OUTPUT"),
        "inouts": _extract_var_block(declaration, "IN_OUT"),
        # Optional: lokale Blöcke, falls gewünscht
        "temps": _extract_var_block(declaration, "TEMP"),
    }

# ---------- Parser für TwinCAT-XML ----------
def parse_tc_pou_anylang(pou_path: Path):
    txt = read_text(pou_path)
    root = ET.fromstring(strip_ns(txt))

    pou = root.find(".//POU")
    name = pou.get("Name") if pou is not None else pou_path.stem

    # Typ (Program / FunctionBlock / Function)
    ptype = (pou.get("POUType") if pou is not None else "") or ""
    decl_node = root.find(".//Declaration")
    declaration = (decl_node.text or "").strip() if decl_node is not None else ""
    if not ptype and declaration:
        m = re.match(r"\s*(PROGRAM|FUNCTION_BLOCK|FUNCTION)\b", declaration, re.I)
        ptype = (m.group(1).title().replace("_", "") if m else "")

    impl_node = root.find(".//Implementation")
    lang_tag, impl_text = detect_impl_lang(impl_node)

    io = extract_io_from_declaration(declaration) if declaration else {"inputs":[], "outputs":[], "inouts":[], "temps":[]}

    return {
        "kind": "POU",
        "name": name,
        "pou_type": ptype,                  # Program | FunctionBlock | Function
        "implementation_lang": lang_tag,    # ST | FBD | LD | SFC | IL | NWL | None
        "declaration": declaration,
        "implementation": impl_text,        # bei FBD/LD meist leer (grafisch)
        "io": io,
        "file": str(pou_path)
    }

def parse_tc_dut(dut_path: Path):
    txt = read_text(dut_path)
    root = ET.fromstring(strip_ns(txt))
    dut = root.find(".//DUT")
    name = dut.get("Name") if dut is not None else dut_path.stem
    # Typ (STRUCT/ENUM/ALIAS/UNION) steckt i. d. R. in der Declaration
    decl_node = root.find(".//Declaration")
    declaration = (decl_node.text or "").strip() if decl_node is not None else ""
    # heuristischer dut_kind
    dut_kind = ""
    m = re.match(r"\s*(TYPE\s+)?(STRUCT|ENUM|UNION|ALIAS)\b", declaration, re.I)
    if m:
        dut_kind = m.group(2).upper()
    return {
        "kind": "DUT",
        "name": name,
        "dut_kind": dut_kind,
        "declaration": declaration,
        "file": str(dut_path)
    }

def parse_tc_gvl(gvl_path: Path):
    txt = read_text(gvl_path)
    root = ET.fromstring(strip_ns(txt))
    gvl = root.find(".//GVL")
    name = gvl.get("Name") if gvl is not None else gvl_path.stem
    decl_node = root.find(".//Declaration")
    declaration = (decl_node.text or "").strip() if decl_node is not None else ""
    # Variablen in GVL stehen üblicherweise in VAR_GLOBAL ... END_VAR
    globals_ = _extract_var_block(declaration, "GLOBAL")
    return {
        "kind": "GVL",
        "name": name,
        "declaration": declaration,
        "globals": globals_,
        "file": str(gvl_path)
    }

def parse_tc_vis(vis_path: Path):
    """
    VISU-Metadaten aus .TcVis (Seitenname). Struktur ist XML; wir lesen den Wurzelknoten.
    """
    try:
        txt = read_text(vis_path)
        root = ET.fromstring(strip_ns(txt))
        vis = root.find(".//Visualization")
        name = (vis.get("Name") if vis is not None else None) or vis_path.stem
    except Exception:
        name = vis_path.stem
    return {
        "kind": "VISU",
        "name": name,
        "file": str(vis_path)
    }

# ---------- .plcproj Utilities ----------
def list_artifacts_in_plcproj(plcproj: Path):
    """
    Sucht referenzierte .TcPOU/.TcDUT/.TcGVL/.TcVis und zusätzlich inline-Objekte im .plcproj.
    """
    txt = strip_ns(read_text(plcproj))
    root = ET.fromstring(txt)
    out = []

    # 1) Referenzen in ItemGroups
    for item in root.findall(".//ItemGroup/*"):
        inc = item.get("Include") or ""
        inc_l = inc.lower()
        p = (plcproj.parent / inc).resolve()

        try:
            if inc_l.endswith(".tcpou") and p.exists():
                out.append(parse_tc_pou_anylang(p))
            elif inc_l.endswith(".tcdut") and p.exists():
                out.append(parse_tc_dut(p))
            elif inc_l.endswith(".tcgvl") and p.exists():
                out.append(parse_tc_gvl(p))
            elif inc_l.endswith(".tcvis") and p.exists():
                out.append(parse_tc_vis(p))
        except Exception as e:
            print(f"⚠️ Fehler beim Parsen {p}: {e}")

    # 2) Inline-POUs/GVLs/DUTs (falls Multiple Project Files nicht aktiv war)
    for pou in root.findall(".//POU"):
        name = pou.get("Name") or ""
        ptype = pou.get("POUType") or ""
        decl = pou.find(".//Declaration")
        impl = pou.find(".//Implementation")
        lang_tag, impl_text = detect_impl_lang(impl)
        declaration = (decl.text or "").strip() if decl is not None else ""
        out.append({
            "kind": "POU",
            "name": name,
            "pou_type": ptype,
            "implementation_lang": lang_tag,
            "declaration": declaration,
            "implementation": impl_text,
            "io": extract_io_from_declaration(declaration) if declaration else {"inputs":[], "outputs":[], "inouts":[], "temps":[]},
            "file": str(plcproj) + " (inline)"
        })
    for gvl in root.findall(".//GVL"):
        name = gvl.get("Name") or ""
        decl = gvl.find(".//Declaration")
        declaration = (decl.text or "").strip() if decl is not None else ""
        out.append({
            "kind": "GVL",
            "name": name,
            "declaration": declaration,
            "globals": _extract_var_block(declaration, "GLOBAL"),
            "file": str(plcproj) + " (inline)"
        })
    for dut in root.findall(".//DUT"):
        name = dut.get("Name") or ""
        decl = dut.find(".//Declaration")
        declaration = (decl.text or "").strip() if decl is not None else ""
        m = re.match(r"\s*(TYPE\s+)?(STRUCT|ENUM|UNION|ALIAS)\b", declaration, re.I)
        dut_kind = m.group(2).upper() if m else ""
        out.append({
            "kind": "DUT",
            "name": name,
            "dut_kind": dut_kind,
            "declaration": declaration,
            "file": str(plcproj) + " (inline)"
        })
    # VISUs sind selten inline; falls vorhanden:
    for vis in root.findall(".//Visualization"):
        name = vis.get("Name") or ""
        out.append({
            "kind": "VISU",
            "name": name,
            "file": str(plcproj) + " (inline)"
        })

    return out

def find_tsprojs_in_sln(sln_path: Path):
    txt = read_text(sln_path)
    tsprojs = []
    for m in re.finditer(r'Project\(".*?"\)\s=\s*".*?",\s*"(.*?)"', txt):
        rel = m.group(1)
        if rel.lower().endswith(".tsproj"):
            tsprojs.append((sln_path.parent / rel).resolve())
    return tsprojs

def find_plcprojs_near(tsproj: Path):
    return list(tsproj.parent.rglob("*.plcproj"))

# ---------- Pfad zu DEINER SLN ----------
sln_path = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\v1.0_Tisch01_Beckhoff_I4.0.sln")

# ---------- Sammeln ----------
tsprojs = find_tsprojs_in_sln(sln_path)
plcprojs = []
for ts in tsprojs:
    plcprojs.extend(find_plcprojs_near(ts))
plcprojs = sorted(set(plcprojs))

all_objs = []
for pp in plcprojs:
    try:
        all_objs.extend(list_artifacts_in_plcproj(pp))
    except Exception as e:
        print(f"⚠️ Fehler beim Parsen {pp}: {e}")

# ---------- Auswertung ----------
kinds = Counter([o.get("kind") for o in all_objs])
print("Objekt-Typen:")
for k, v in kinds.items():
    print(f"  {k}: {v}")

# Beispiele je Typ
by_kind = defaultdict(list)
for o in all_objs:
    by_kind[o.get("kind")].append(o)

print("\nBeispiele je Typ:")
for kind, items in by_kind.items():
    print(f"\n== {kind} ==")
    for o in items[:5]:  # max 5 Beispiele
        if kind == "POU":
            io = o.get("io", {})
            io_sum = f"in={len(io.get('inputs',[]))}, out={len(io.get('outputs',[]))}, inout={len(io.get('inouts',[]))}"
            print(f"- {o['name']}  [{o.get('pou_type','?')}/{o.get('implementation_lang') or '—'}]  IO({io_sum}) -> {o['file']}")
        elif kind == "DUT":
            print(f"- {o['name']}  [{o.get('dut_kind') or '—'}] -> {o['file']}")
        else:
            print(f"- {o['name']} -> {o['file']}")

# Nur POUs mit ST-Implementation zeigen + deren IO-Variablen
st_pous = [o for o in all_objs if o.get("kind")=="POU" and (o.get("implementation_lang") or "").upper()=="ST"]
print(f"\nSummary: PLCProjs={len(plcprojs)}, Objects={len(all_objs)}, ST-POUs={len(st_pous)}")

# Optional: Dateien schreiben (JSONs neben der SLN)
out_base = sln_path.with_suffix("")
Path(str(out_base) + "_objects.json").write_text(json.dumps(all_objs, indent=2, ensure_ascii=False), encoding="utf-8")
Path(str(out_base) + "_pous_st.json").write_text(json.dumps(st_pous, indent=2, ensure_ascii=False), encoding="utf-8")
print("\nExport:")
print(" -", str(out_base) + "_objects.json")
print(" -", str(out_base) + "_pous_st.json")

# Beispielhafte Ausgabe der IO-Listen und ST-Implementierung (gekürzt) für die ersten 3 ST-POUs
print("\n--- ST-POU IO-Details (erste 3) ---")
for o in st_pous[:3]:
    print(f"\nPOU {o['name']} ({o.get('pou_type','?')})")
    io = o["io"]
    for label, lst in [("VAR_INPUT", io["inputs"]), ("VAR_OUTPUT", io["outputs"]), ("VAR_IN_OUT", io["inouts"])]:
        print(f"  {label}:")
        for v in lst:
            addr = f" @ {v['address']}" if v['address'] else ""
            init = f" := {v['init']}" if v['init'] else ""
            print(f"    - {v['name']}: {v['type']}{addr}{init}")
    # ST-Code (falls vorhanden) leicht gekürzt
    impl = (o.get("implementation") or "").strip()
    if impl:
        preview = impl if len(impl) < 800 else impl[:800] + "\n... [gekürzt] ..."
        print("\n  ST-Implementation (Preview):\n" + preview)


Objekt-Typen:
  GVL: 7
  POU: 50

Beispiele je Typ:

== GVL ==
- GVL -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\GVLs\GVL.TcGVL
- GVL_Diagnose -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\GVLs\GVL_Diagnose.TcGVL
- GVL_HRL -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\GVLs\GVL_HRL.TcGVL
- GVL_MBS -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\GVLs\GVL_MBS.TcGVL
- GVL_SST -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\GVLs\GVL_SST.TcGVL

== POU ==
- AutomaticColorDetection_nichtfertig  [FunctionBlock/NWL]  IO(in=7, out=3, inout=0) -> C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\Test1\SPS_Demonstrator\POUs\AtomicSkillsSchablonen\AutomaticColorDetection_nichtfertig.TcPOU
- AxisControl_MultipleInputSensors  [FunctionBlock/NWL]  IO(in=10,

In [9]:
import json, pathlib

# 1) JSON einlesen
json_path = pathlib.Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\v1.0_Tisch01_Beckhoff_I4.0_objects.json")
with json_path.open(encoding="utf-8") as f:
    objects = json.load(f)

plc_names = set()

# 2) Für jedes Artefakt den Elternordner durchsuchen, bis .plcproj gefunden wird
for obj in objects:
    fpath = pathlib.Path(obj["file"])
    # inline-Einträge haben " (inline)" am Ende, deshalb originalen Pfad extrahieren
    try:
        fpath = pathlib.Path(fpath.as_posix().split(" (inline)")[0])
    except Exception:
        pass
    for parent in fpath.parents:
        for plcproj in parent.glob("*.plcproj"):
            plc_names.add(plcproj.stem)
            break
        else:
            continue
        break

# 3) Aus PLC-Namen Lookup-Pfade bauen
lookup_paths = [f"TIPC^{name}^{name} Project" for name in sorted(plc_names)]
print("Gefundene PLC-Projekte:", lookup_paths)


Gefundene PLC-Projekte: ['TIPC^SPS_Demonstrator^SPS_Demonstrator Project']


In [10]:
import json, pathlib, win32com.client as com

# ----------------- PLC-Namen aus der JSON (siehe oben) -----------------
json_path = pathlib.Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\v1.0_Tisch01_Beckhoff_I4.0_objects.json")
with open(json_path, encoding="utf-8") as f:
    objects = json.load(f)

plc_names = set()
for obj in objects:
    fpath = pathlib.Path(obj["file"].split(" (inline)")[0])
    # nach *.plcproj im Elternverzeichnis suchen
    for parent in fpath.parents:
        for plcproj in parent.glob("*.plcproj"):
            plc_names.add(plcproj.stem)
            break
        else:
            continue
        break

lookup_paths = [f"TIPC^{name}^{name} Projekt" for name in sorted(plc_names)]
# -----------------------------------------------------------------------

# TwinCAT-Projekt öffnen
sln_path   = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\v1.0_Tisch01_Beckhoff_I4.0.sln"
export_xml = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml"

dte = com.Dispatch("TcXaeShell.DTE.17.0")  # Version ggf. anpassen
dte.SuppressUI = False
dte.MainWindow.Visible = True
solution = dte.Solution
solution.Open(sln_path)

# SystemManager aus dem TwinCAT-Projekt holen:contentReference[oaicite:1]{index=1}
project = solution.Projects.Item(1)  # erstes TwinCAT-Projekt
sys_mgr = project.Object             # SystemManager-Instanz

# passenden PLC-Knoten finden
plc_project = None
for path in lookup_paths:
    try:
        plc_project = sys_mgr.LookupTreeItem(path)
        break
    except Exception:
        continue

if plc_project is None:
    raise RuntimeError(f"Kein PLC-Projekt gefunden – prüfen Sie Namen: {plc_names}")

# PLCopen-Export ausführen:contentReference[oaicite:2]{index=2}
import_export = plc_project          # impliziter Cast zu ITcPlcIECProject
import_export.PlcOpenExport(export_xml, "")
print("XML-Export erstellt:", export_xml)


XML-Export erstellt: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml


In [16]:
from collections import defaultdict
from pathlib import Path
import json
import xml.etree.ElementTree as ET

NS = {'ns': 'http://www.plcopen.org/xml/tc6_0200'}

def parse_io_vars(pou):
    """Liefert Listen der deklarierten Inputs und Outputs aus der Interface-Sektion eines POU."""
    inputs, outputs = [], []
    interface = pou.find('ns:interface', NS)
    if interface is not None:
        input_vars = interface.find('ns:inputVars', NS)
        if input_vars is not None:
            for var in input_vars.findall('ns:variable', NS):
                name = var.attrib.get('name')
                if name:
                    inputs.append(name)
        output_vars = interface.find('ns:outputVars', NS)
        if output_vars is not None:
            for var in output_vars.findall('ns:variable', NS):
                name = var.attrib.get('name')
                if name:
                    outputs.append(name)
    return inputs, outputs

def build_node_mapping(fbd):
    """Erzeugt ein Dictionary localId -> externer Ausdruck für inVariable/outVariable-Knoten."""
    node_expr = {}
    for inv in fbd.findall('ns:inVariable', NS):
        lid = inv.get('localId')
        expr = inv.find('ns:expression', NS)
        if lid and expr is not None and expr.text:
            node_expr[lid] = expr.text.strip()
    for outv in fbd.findall('ns:outVariable', NS):
        lid = outv.get('localId')
        expr = outv.find('ns:expression', NS)
        if lid and expr is not None and expr.text:
            node_expr[lid] = expr.text.strip()
    return node_expr

def extract_call_blocks(fbd, pou_names_set, node_map):
    """Sammelt die Aufrufe von Unterprogrammen (block.typeName in pou_names_set) und deren I/O-Mapping."""
    calls = []
    for block in fbd.findall('ns:block', NS):
        type_name = block.get('typeName')
        if type_name and type_name in pou_names_set:
            call_info = {
                'SubNetwork_Name': type_name,
                'instanceName': block.get('instanceName'),
                'inputs': [],
                'outputs': [],
            }
            # Eingänge der Subfunktion auslesen
            for var in block.findall('ns:inputVariables/ns:variable', NS):
                formal = var.get('formalParameter')
                ext = None
                cpin = var.find('ns:connectionPointIn', NS)
                if cpin is not None:
                    conn = cpin.find('ns:connection', NS)
                    if conn is not None:
                        ref = conn.get('refLocalId')
                        if ref:
                            ext = node_map.get(ref, f'localId:{ref}')
                call_info['inputs'].append({'internal': formal, 'external': ext})
            # Ausgänge der Subfunktion auslesen
            for var in block.findall('ns:outputVariables/ns:variable', NS):
                formal = var.get('formalParameter')
                ext = None
                cpout = var.find('ns:connectionPointOut', NS)
                if cpout is not None:
                    expr = cpout.find('ns:expression', NS)
                    if expr is not None and expr.text:
                        ext = expr.text.strip()
                    else:
                        conn = cpout.find('ns:connection', NS)
                        if conn is not None:
                            ref = conn.get('refLocalId')
                            if ref:
                                ext = node_map.get(ref, f'localId:{ref}')
                call_info['outputs'].append({'internal': formal, 'external': ext})
            calls.append(call_info)
    return calls

def map_pou_io_to_external(pou, node_map):
    """
    Ordnet deklarierten Inputs/Outputs eines POU den externen Variablennamen zu,
    sofern sie in den in/out-Variablen des FBD-Blocks erscheinen.
    """
    inputs, outputs = parse_io_vars(pou)
    mapped_inputs = []
    mapped_outputs = []
    # Reverse-Mapping: Wenn das Ausdrucks-Suffix dem internen Namen entspricht, wird es als externe Variable verwendet.
    for inp in inputs:
        ext = None
        for expr in node_map.values():
            if expr.split('.')[-1] == inp:
                ext = expr
                break
        mapped_inputs.append({'internal': inp, 'external': ext})
    for out in outputs:
        ext = None
        for expr in node_map.values():
            if expr.split('.')[-1] == out:
                ext = expr
                break
        mapped_outputs.append({'internal': out, 'external': ext})
    return mapped_inputs, mapped_outputs

def analyze_plcopen(xml_path):
    """Analysiert die PLCopen-XML und erzeugt eine Liste aus Programminformationen und Subnetz-Aufrufen."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    pou_names = {p.attrib.get('name') for p in root.findall('.//ns:pou', NS)}
    result = []
    for pou in root.findall('.//ns:pou', NS):
        name = pou.attrib.get('name')
        fbd = pou.find('.//ns:FBD', NS)
        node_map = build_node_mapping(fbd) if fbd is not None else {}
        inputs, outputs = parse_io_vars(pou)
        mapped_inputs, mapped_outputs = ([], [])
        if fbd is not None:
            mapped_inputs, mapped_outputs = map_pou_io_to_external(pou, node_map)
        else:
            mapped_inputs = [{'internal': n, 'external': None} for n in inputs]
            mapped_outputs = [{'internal': n, 'external': None} for n in outputs]
        subcalls = extract_call_blocks(fbd, pou_names, node_map) if fbd is not None else []
        result.append({
            'Programm_Name': name,
            'inputs': mapped_inputs,
            'outputs': mapped_outputs,
            'subcalls': subcalls
        })
    return result

# Beispielaufruf:
xml_file = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml")
mapping = analyze_plcopen(xml_file)
with open(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\program_io_with_mapping.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)


In [23]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import defaultdict

# -- Pfade anpassen --
json_path = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\program_io_with_mapping.json")
xml_path  = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml")

# JSON-Datei einlesen (Programme mit Ein-/Ausgängen und Subcalls)
data    = json.loads(json_path.read_text(encoding="utf-8"))
pou_map = {item["Programm_Name"]: item for item in data}

# XML-Datei einlesen und Variablen + physische Adresse auslesen
root = ET.parse(xml_path).getroot()
NS   = {"ns": "http://www.plcopen.org/xml/tc6_0200",
        "html": "http://www.w3.org/1999/xhtml"}

var_doc = {}
hardware_inputs  = set()
hardware_outputs = set()
for var in root.findall(".//ns:variable", NS):
    name = var.attrib.get("name")
    doc_elem = var.find(".//html:xhtml", NS)
    if doc_elem is not None and doc_elem.text:
        doc = doc_elem.text.strip()
        var_doc[name] = doc
        if doc.startswith(("xDI", "udiDI")):
            hardware_inputs.add(name)
        elif doc.startswith(("xDO", "udiDO")):
            hardware_outputs.add(name)

# Hilfsfunktion: letzten Namensbestandteil (GVL_X.Y -> Y)
def base_name(expr: str) -> str:
    return expr.split(".")[-1] if expr else ""

# Call-Graph aufbauen
call_graph    = defaultdict(list)
reverse_graph = defaultdict(list)
for entry in data:
    caller = entry["Programm_Name"]
    for sub in entry.get("subcalls", []):
        callee = sub["SubNetwork_Name"]
        call_graph[caller].append(callee)
        reverse_graph[callee].append(caller)

# Rekursive Berechnung der aggregierten Hardware-Ein-/Ausgänge (inkl. Subcalls)
memo_out, memo_in = {}, {}

def get_hw_outputs_agg(pou_name):
    if pou_name in memo_out:
        return memo_out[pou_name]
    entry = pou_map.get(pou_name)
    if not entry:
        memo_out[pou_name] = set()
        return set()
    outputs = set()
    # direkte Ausgänge
    for out in entry["outputs"]:
        ext = out.get("external")
        if ext and base_name(ext) in hardware_outputs:
            outputs.add(ext)
    # Subcalls berücksichtigen
    for sub in entry.get("subcalls", []):
        sub_name = sub["SubNetwork_Name"]
        outputs |= get_hw_outputs_agg(sub_name)
        for sop in sub["outputs"]:
            ext = sop.get("external")
            if ext and base_name(ext) in hardware_outputs:
                outputs.add(ext)
    memo_out[pou_name] = outputs
    return outputs

def get_hw_inputs_agg(pou_name):
    if pou_name in memo_in:
        return memo_in[pou_name]
    entry = pou_map.get(pou_name)
    if not entry:
        memo_in[pou_name] = set()
        return set()
    inputs = set()
    # direkte Eingänge
    for inp in entry["inputs"]:
        ext = inp.get("external")
        if ext and base_name(ext) in hardware_inputs:
            inputs.add(ext)
    # Subcalls berücksichtigen
    for sub in entry.get("subcalls", []):
        sub_name = sub["SubNetwork_Name"]
        inputs |= get_hw_inputs_agg(sub_name)
        for sp in sub["inputs"]:
            ext = sp.get("external")
            if ext and base_name(ext) in hardware_inputs:
                inputs.add(ext)
    memo_in[pou_name] = inputs
    return inputs

# Ergebnisstruktur aufbauen
result = []
for pou_name, entry in pou_map.items():
    # direkte Hardware-Inputs/-Outputs und Control-Variablen
    direct_hw_inputs, direct_hw_outputs = [], []
    control_inputs, control_outputs    = [], []
    for inp in entry["inputs"]:
        ext      = inp.get("external")
        internal = inp.get("internal")
        if ext and base_name(ext) in hardware_inputs:
            addr = var_doc.get(base_name(ext))
            direct_hw_inputs.append({"variable": ext, "address": addr})
        else:
            # kein oder kein Hardware-Mapping -> Kontroll-Eingang
            control_inputs.append(internal)
    for out in entry["outputs"]:
        ext      = out.get("external")
        internal = out.get("internal")
        if ext and base_name(ext) in hardware_outputs:
            addr = var_doc.get(base_name(ext))
            direct_hw_outputs.append({"variable": ext, "address": addr})
        else:
            # kein oder kein Hardware-Mapping -> Kontroll-Ausgang
            control_outputs.append(internal)

    # aggregierte Hardware-Inputs/-Outputs (rekursiv über Subcalls)
    aggregated_inputs  = []
    for ext in sorted(get_hw_inputs_agg(pou_name)):
        addr = var_doc.get(base_name(ext))
        aggregated_inputs.append({"variable": ext, "address": addr})
    aggregated_outputs = []
    for ext in sorted(get_hw_outputs_agg(pou_name)):
        addr = var_doc.get(base_name(ext))
        aggregated_outputs.append({"variable": ext, "address": addr})

    # aufrufende und aufgerufene Programme
    callers  = sorted(reverse_graph.get(pou_name, []))
    subcalls = sorted(call_graph.get(pou_name, []))

    result.append({
        "Program_Name": pou_name,
        "Hardware_Inputs": aggregated_inputs,      # alle physischen Eingänge (direkt + Subcalls)
        "Caller_Programs": callers,                # wer ruft dieses POU auf?
        "Control_Inputs": control_inputs,          # Eingänge ohne Hardware-Bezug
        "Control_Outputs": control_outputs,        # Ausgänge ohne Hardware-Bezug
        "Hardware_Outputs": aggregated_outputs,    # alle physischen Ausgänge (direkt + Subcalls)
        "Subcall_Programs": subcalls               # welche Subprogramme ruft dieses POU auf?
    })

# Ausgabe als JSON speichern (optional)
# out_path = Path(r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\program_hw_io_chain.json")
# out_path.write_text(json.dumps(result, ensure_ascii=False, indent=2), encoding="utf-8")

# Beispiel: Nur ausgewählte Programme anzeigen
for prog in result:
    if prog["Program_Name"] in ["HRL_SkillSet", "SST_SkillSet", "MBS_SkillSet", "VSG_SkillSet"]:
        print(f"{prog['Program_Name']}:")
        print("  Hardware_Inputs:", prog["Hardware_Inputs"])
        print("  Caller_Programs:", prog["Caller_Programs"])
        print("  Control_Inputs:", prog["Control_Inputs"])
        print("  Control_Outputs:", prog["Control_Outputs"])
        print("  Hardware_Outputs:", prog["Hardware_Outputs"])
        print("  Subcall_Programs:", prog["Subcall_Programs"])
        print()


HRL_SkillSet:
  Hardware_Inputs: [{'variable': 'GVL_HRL.HRL_Enc_horizontal_I1', 'address': 'udiDI%41.0_A2_5'}, {'variable': 'GVL_HRL.HRL_Enc_horizontal_I2', 'address': 'udiDI%47.0_A2_5'}, {'variable': 'GVL_HRL.HRL_Enc_vertikal_I1', 'address': 'udiDI%53.0_A2_5'}, {'variable': 'GVL_HRL.HRL_Enc_vertikal_I2', 'address': 'udiDI%59.0_A2_5'}, {'variable': 'GVL_HRL.HRL_LS_aussen', 'address': 'xDI%136.2_A2_1'}, {'variable': 'GVL_HRL.HRL_LS_innen', 'address': 'xDI%136.1_A2_1'}, {'variable': 'GVL_HRL.HRL_Ref_Taster_Ausleger_hinten', 'address': 'xDI%136.7_A2_2'}, {'variable': 'GVL_HRL.HRL_Ref_Taster_Ausleger_vorne', 'address': 'xDI%136.6_A2_2'}, {'variable': 'GVL_HRL.HRL_Ref_Taster_horizontal', 'address': 'xDI%136.0_A2_2'}, {'variable': 'GVL_HRL.HRL_Ref_Taster_vertikal', 'address': 'xDI%136.3_A2_2'}]
  Caller_Programs: []
  Control_Inputs: ['ResetButton', 'StartButton', 'ConfirmationButton']
  Control_Outputs: []
  Hardware_Outputs: [{'variable': 'GVL_HRL.HRL_MOT_Ausleger_rueckwaerts', 'address': 

In [ ]:
r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\export.xml"